In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_pickle("../input/consumption_day.pkl")

print(df.shape)
df.head()

(3248, 366)


,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,...,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.030,5.397,0.0,4.818,3.931,0.0,4.503,0.0,5.129,5.395
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.101,14.327,0.0,14.936,16.174,0.0,24.618,0.0,15.167,11.751
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.201,7.320,0.0,7.384,14.425,0.0,17.705,0.0,8.966,4.633
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.833,12.477,0.0,10.974,19.646,0.0,23.993,0.0,15.841,14.452
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.440,35.538,0.0,8.351,9.957,0.0,25.871,0.0,46.274,16.901


In [3]:
start = datetime(2017, 1, 1)
end = datetime(2017, 12, 31)
date_range = pd.date_range(start, end)

df_train = pd.DataFrame(columns=["meter_id", "date", "meter_reading"])

for meter_idx in tqdm(df["meter_id"].values):
    
    df_meter_idx = pd.DataFrame(columns=["meter_id", "date", "meter_reading"])
    df_meter_idx["date"] = date_range
    df_meter_idx["meter_id"] = meter_idx
    df_meter_idx["meter_reading"] = df[df.meter_id == meter_idx].stack().values[1:].astype(float)

    df_train = pd.concat([df_train, df_meter_idx]).reset_index(drop=True)

df_train = df_train.sort_values(["meter_id", "date"]).reset_index(drop=True)
df_train["meter_reading"] = df_train["meter_reading"].round(3)
print(df_train.shape)
df_train.head()


(1185520, 3)


,meter_id,date,meter_reading
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-01-01,0.0
1,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-01-02,0.0
2,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-01-03,0.0
3,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-01-04,0.0
4,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-01-05,0.0


In [4]:
# REMOVE missing data - remove data if 3 days sensor mean readings are 0
df_train["meter_reading_rolling_3"] = df_train.groupby(["meter_id"])["meter_reading"].rolling(3).mean().fillna(-1).values
df_train = df_train[df_train.meter_reading_rolling_3 > 1e-3].reset_index(drop=True)
df_train.drop(["meter_reading_rolling_3"], axis=1, inplace=True)
print(df_train.shape)
df_train.head()

(636598, 3)


,meter_id,date,meter_reading
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-01,4.143
1,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-02,4.116
2,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-03,4.101
3,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-04,4.124
4,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-05,4.111


In [5]:
### most of below are from the check_outliers.ipynb notesbook

In [6]:
# CLUSTER 0: 0-270
#257 (all zeros)
df_train = df_train.query('not (meter_id == "0x81fa8eddb2b09393d3719984ca5520cb50f45efd")')

In [7]:
# CLUSTER 1: 271-541
#377
df_train = df_train.query('not (meter_id == "0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471" & date < "2017-12-20")')

#515
df_train = df_train.query('not (meter_id == "0x423fa805ddb0cba9bdb4460f9a78540287eefd0e")')

In [8]:
# CLUSTER 2: 542-812
#734
df_train = df_train.query('not (meter_id == "0xc84773d3755ad01dadad61d45008ddf91cb1f17f" & date < "2017-12-1")')

#775
df_train = df_train.query('not (meter_id == "0xd23ef71e350e2d1c196c4e8c86b0366f24625f27")')

#785
df_train = df_train.query('not (meter_id == "0x091e6452a3b5d6f5ed0364d4832757c726eb59a5" & date < "2017-11-1")')

In [9]:
# CLUSTER 3: 813-1083
#823
df_train = df_train.query('not (meter_id == "0xe405bb258bb4afff752ea6b745974fbabe13681b" & date < "2017-12-1")')

#873
df_train = df_train.query('not (meter_id == "0xd1dd81631b89047ee2ed4eb83f88b011a0beee96" & date < "2017-11-1")')

#947
df_train = df_train.query('not (meter_id == "0x8af6a4d9bf889f310b897dddfac2d669b945c548" & date < "2017-12-1")')

#993
df_train = df_train.query('not (meter_id == "0xf226527dbcb9b2afdbdb5d63e7c27d8bef036941")')

#1025
df_train = df_train.query('not (meter_id == "0x669a5529a8c912ae5df50d05b9b1f12a623304c1")')

#1048
df_train = df_train.query('not (meter_id == "0x0cccf4ed9c731495f4156884240dd4d74e471ba2" & date < "2017-11-15")')

#1061
df_train = df_train.query('not (meter_id == "0x928639442e6f52a387c1f2930d17f9b95910ce8c" & date < "2017-12-10")')

In [10]:
# CLUSTER 4: 1084-1354
#1091
df_train = df_train.query('not (meter_id == "0xe41c230484378b2312fbf1fc99d289e0b51a3c52")')

#1150
df_train = df_train.query('not (meter_id == "0xa0a3003a10a350195ebe503e0899ebeaccee78ae")')

#1245
df_train = df_train.query('not (meter_id == "0xd9583a22cacb4a8e859bc9f33b54612c86fe7ce2" & date < "2017-11-1")')

#1251
df_train = df_train.query('not (meter_id == "0x7900a0fa75b06d938464f776ded709b8318c320c" & date < "2017-11-1")')

#1290
df_train = df_train.query('not (meter_id == "0x087e8414ef3e98ee3878b0f764025f85522cb3e6" & date < "2017-12-1")')

#1342
df_train = df_train.query('not (meter_id == "0x95fc7e54336e87feed25423b86d9ea92e47432dc" & date < "2017-12-1")')

#1350
df_train = df_train.query('not (meter_id == "0x92ed8eeea2e07d87de0d3a07f8592bbefc036267" & date < "2017-11-1")')

#1351
df_train = df_train.query('not (meter_id == "0xa9635068c6db75ffef8332f2a56a32e5a75ac21c" & date < "2017-12-1")')

In [11]:
# CLUSTER 5: 1355-1625
#1360
df_train = df_train.query('not (meter_id == "0x800c9648a84c44cf1a1b0a0ada6283495a69fe8b")')

#1419
df_train = df_train.query('not (meter_id == "0xd4f466b4dd0cff2bdec56816b5e8b78309d8a525" & date < "2017-11-1")')

#1435
df_train = df_train.query('not (meter_id == "0x0a7d02ab9cf4a3dd1a21f6940b5f8ad91fcbd13a" & date > "2017-9-1")')

#1535
df_train = df_train.query('not (meter_id == "0x4d2d19dd444313b0f17fe087ee5cbb80fd53d682" & date < "2017-9-1")')

#1545
df_train = df_train.query('not (meter_id == "0x59799e9835ff86e7926d43592c9fa93415a8b754" & date < "2017-12-1")')

#1596
df_train = df_train.query('not (meter_id == "0xf0addcdd2c402d4a2f8a7a7a95c7d246d1c59b63" & date > "2017-9-1")')

#1608
df_train = df_train.query('not (meter_id == "0x0cf2db66a7e366ebb5cb9866e2b2333cdb42b75c" & date < "2017-12-1")')

In [12]:
# CLUSTER 6: 1626-1896
#1671
df_train = df_train.query('not (meter_id == "0x9b7d38dac67f0cac0b721d97b86d49a531ceafd7" & date < "2017-9-1")')

#1797
df_train = df_train.query('not (meter_id == "0x024dde4050a50ee78153c99ea37fda1326fc6468" & date < "2017-10-1")')

#1816
df_train = df_train.query('not (meter_id == "0x8594629a224eed965ab0222613eefb88781ac784" & date < "2017-12-1")')

#1887
df_train = df_train.query('not (meter_id == "0x5c74fe4757ca70cf3c8e2ea491db1ae420649299" & date < "2017-12-1")')

#1894
df_train = df_train.query('not (meter_id == "0xfc287868b286c46de7210470f2d3e2a1b5334ce3" & date < "2017-11-1")')

In [13]:
# CLUSTER 7: 1897-2167
#1922
df_train = df_train.query('not (meter_id == "0xc0ce1f0aa80acb8c9ede1156b53bdc1b3faec297" & date < "2017-8-1")')

#1953
df_train = df_train.query('not (meter_id == "0xc73b4d676b98a29a4c3c4ed78c8f1d208b96fa11" & date < "2017-11-1")')

#1966
df_train = df_train.query('not (meter_id == "0x4dd7140b3e4911a17ab45b6c4d83cda08caa1712" & date < "2017-11-1")')

#2026
df_train = df_train.query('not (meter_id == "0x0d1e3f2881f92da5f0c2250854ace74b82cd6aae" & date < "2017-11-10")')

#2035
df_train = df_train.query('not (meter_id == "0xdc06d14af4048e4b4b38b8119c77dcce785c1aa5" & date < "2017-7-1")')

#2042
df_train = df_train.query('not (meter_id == "0x0bb3e76a57e589d8e2a930aa868c225f5347fc71" & date < "2017-11-1")')

#2060
df_train = df_train.query('not (meter_id == "0xca77a75d9ce22d916358c9335c35f2b0ad5581ba" & date < "2017-11-15")')

#2080
df_train = df_train.query('not (meter_id == "0xe2154c841409bba5b83d208c83c33938d1a02474")')

#2084
df_train = df_train.query('not (meter_id == "0x57f9a238a2eb176e3e8a34d5be35adfca6341c6f" & date < "2017-11-15")')

#2151
df_train = df_train.query('not (meter_id == "0xe5c7a9422ea390997565ae6f037383ff79cf9daa" & date < "2017-9-1")')

In [14]:
# CLUSTER 8: 2168-2437
#2271
df_train = df_train.query('not (meter_id == "0x1527c03f99b223369214530034b4ba6d006097b7" & date < "2017-7-1")')

#2286
df_train = df_train.query('not (meter_id == "0xfd1a269f4667f21684486b89f3c0fa7e2fa03a5d" & date > "2017-7-1")')

#2290
df_train = df_train.query('not (meter_id == "0xca8dbfb4bfc0f430b102688b4f5d90c7c64feeab" & date < "2017-9-1")')

#2311
df_train = df_train.query('not (meter_id == "0xf5df47623993f1e0c327bad42a1c3289232d31ae" & date < "2017-10-1")')

#2424
df_train = df_train.query('not (meter_id == "0x240e5e22734a44a174b7dabcf1ea00d70d9ec168" & date < "2017-11-1")')

In [15]:
# CLUSTER 9: 2438-2707
#2470
df_train = df_train.query('not (meter_id == "0x06f762afdc7b9efa97a076ec3cf1c60f17f939d8" & meter_reading > 20)')

#2531
df_train = df_train.query('not (meter_id == "0x6ac8944f86574cda6f77357c80b43115068132c2" & date < "2017-5-1")')

#2642
df_train = df_train.query('not (meter_id == "0x5e38e74c0bb5a70dfd4cdfb99f9daaf94e3c2176" & date < "2017-5-1")')

#2669
df_train = df_train.query('not (meter_id == "0xf68047ef8dd34bbafc9e87c54a27b4e416b337f1" & date < "2017-9-1")')

In [16]:
# CLUSTER 10: 2708-2977
#2711
df_train = df_train.query('not (meter_id == "0xdd6f5700bd227b17f00c19564563094661450620")')

#2907
df_train = df_train.query('not (meter_id == "0x843fed762825bfdc1f8f07349d425fd5c7b4cd1e" & date > "2017-12-1")')

#2955
df_train = df_train.query('not (meter_id == "0x463d557082956a82ac9cc68ef5ccf15a0a7ebda8" & date < "2017-9-1")')

#2975
df_train = df_train.query('not (meter_id == "0xbfa2344ecf7cc72a62a3a70599c455dac98c32e3" & date < "2017-7-1")')

In [17]:
# CLUSTER 11: 2978-3247
#3058
df_train = df_train.query('not (meter_id == "0x866f47864ad46756755708467a8f20fe130fcf19" & date < "2017-7-1")')

#3110
df_train = df_train.query('not (meter_id == "0x26460be217deb4c1898e771f96fff662b4cf07c1" & date < "2017-7-1")')

#3139
df_train = df_train.query('not (meter_id == "0x6e7987cc630ae6a908a9a87d8543c0397951f01f" & date < "2017-7-1")')

#3211
df_train = df_train.query('not (meter_id == "0x8bffc129d034f2c7526b846fb9ed7eea6b67c8d8" & date < "2017-10-1")')

#3213
df_train = df_train.query('not (meter_id == "0xf69f1323a0b54b37dfe58c89387927758ba94a03" & date < "2017-11-1")')

#3224
df_train = df_train.query('not (meter_id == "0x61c5f4683169ab47eac0bbd439919f2d8ec68dd8" & date < "2017-9-1")')

#3228
df_train = df_train.query('not (meter_id == "0x6b229d0b76a548511fab0a8a0dc6ba37b2b61fdb" & date < "2017-9-1")')

#3230
df_train = df_train.query('not (meter_id == "0x317e1b6bca135656cf58c38e8984b857a772cfb5" & date < "2017-11-1")')

#3241
df_train = df_train.query('not (meter_id == "0x6b6ddc9a73405caab50c8d5575b72288aaa7e844" & date < "2017-4-1")')

#3243
df_train = df_train.query('not (meter_id == "0x7dd7a7b8ee1bec7c44b24f738c752482f6161065" & date < "2017-7-1")')

In [18]:
# get time feature
df_train["month"] = df_train["date"].dt.month.astype(int)
df_train["day_of_month"] = df_train["date"].dt.day.astype(int)
df_train["day_of_week"] = df_train["date"].dt.dayofweek.astype(int)

print(df_train.shape)
df_train.head()

(629924, 6)


,meter_id,date,meter_reading,month,day_of_month,day_of_week
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-01,4.143,8,1,1
1,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-02,4.116,8,2,2
2,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-03,4.101,8,3,3
3,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-04,4.124,8,4,4
4,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-05,4.111,8,5,5


In [19]:
# mean year meter reading
meter_reading_mean = df_train.groupby(["meter_id"])["meter_reading"].mean().rename("meter_reading_mean").round(3)
df_train = df_train.merge(meter_reading_mean, on="meter_id", how="left")
df_train["meter_reading_mean"].fillna(0, inplace=True)
print(f"meter map length is {len(meter_reading_mean)}")
# print(set(df.meter_id) - set(meter_reading_mean.index.values))

print(df_train.shape)
df_train.head()

meter map length is 3238
(629924, 7)


,meter_id,date,meter_reading,month,day_of_month,day_of_week,meter_reading_mean
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-01,4.143,8,1,1,3.241
1,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-02,4.116,8,2,2,3.241
2,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-03,4.101,8,3,3,3.241
3,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-04,4.124,8,4,4,3.241
4,0x0001f1c389823f953b2eaee0a61c33539744da0c,2017-08-05,4.111,8,5,5,3.241


In [20]:
test_start = datetime(2018, 1, 1)
test_end = datetime(2018, 12, 31)
test_date_range = pd.date_range(test_start, test_end)

df_test = pd.DataFrame(columns=["meter_id", "date", "meter_reading"])

for meter_idx in tqdm(df["meter_id"].values):
    
    df_meter_idx = pd.DataFrame(columns=["meter_id", "date", "meter_reading"])

    df_meter_idx["date"] = test_date_range
    df_meter_idx["meter_id"] = meter_idx
    df_meter_idx["meter_reading"] = 0 # target

    df_test = pd.concat([df_test, df_meter_idx]).reset_index(drop=True)

df_test = df_test.sort_values(["meter_id", "date"]).reset_index(drop=True)

print(df_test.shape)
df_test.head()


(1185520, 3)


,meter_id,date,meter_reading
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-01,0
1,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-02,0
2,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-03,0
3,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-04,0
4,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-05,0


In [21]:
df_test["month"] = df_test["date"].dt.month.astype(int)
df_test["day_of_month"] = df_test["date"].dt.day.astype(int)
df_test["day_of_week"] = df_test["date"].dt.dayofweek.astype(int)

print(df_test.shape)
df_test.head()

(1185520, 6)


,meter_id,date,meter_reading,month,day_of_month,day_of_week
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-01,0,1,1,0
1,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-02,0,1,2,1
2,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-03,0,1,3,2
3,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-04,0,1,4,3
4,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-05,0,1,5,4


In [22]:
# mean year meter reading
df_test = df_test.merge(meter_reading_mean, on="meter_id", how="left")
df_test["meter_reading_mean"].fillna(0, inplace=True)

print(df_test.shape)
df_test.head()

(1185520, 7)


,meter_id,date,meter_reading,month,day_of_month,day_of_week,meter_reading_mean
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-01,0,1,1,0,3.241
1,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-02,0,1,2,1,3.241
2,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-03,0,1,3,2,3.241
3,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-04,0,1,4,3,3.241
4,0x0001f1c389823f953b2eaee0a61c33539744da0c,2018-01-05,0,1,5,4,3.241


In [23]:
print(df_train.shape, df_test.shape)

# df_train.to_pickle("../input/df_train_clean.pkl")
# df_test.to_pickle("../input/df_test_clean.pkl")
# print("train/test saved to pickle!")

(629924, 7) (1185520, 7)
